In [18]:
#SET-UP AND INITIALIZING

import networkx as nx
import copy
import json
import re, requests

# List of definitions:

""" Remove street node function"""
def remove_street_node(nodes_dict, edge_dict, street_node):
    #nodes_dict: Nested dictionary with nodes as keys and its attributes(dict) as values
    #edge_dict: Dict, key as Nodes, Values as all nodes connected to Nodes
    #street_node: As list of nodes representing the streets
    
    edge_dict_streetless = edge_dict.copy()
    k = list(nodes_dict.keys())
    for i in k:
        #for every node in the grid
        for street in street_node:
            #for every street nodes in the street_node list

            if street in edge_dict[i]:
                #if a street node lies in the list of edges within edge_dict
                edge_dict_streetless.pop(i, None)
            
            elif street == i:
                #delete itself because it is a street node
                edge_dict_streetless.pop(i, None)

    return edge_dict_streetless

""" Create a Edges dict in the form of Nodes (keys) : 3 Other neighbouring nodes (values)"""
def create_edge_dict(graph):

    for i, n in G.adjacency():
        edge_dict[i] = list(n)
    return edge_dict

""" Find parent of the value in a nested dictionary"""
def find_key(d, value):
    start_list =[]
    for i in value:
        # print("i is", i)
        for k,v in d.items():
            # print("k and v is", k,v)
            if v['name'] == i:
                # print("k is", k)
                start_list.append(i)
    return start_list

""" Gives the value of type key of a node"""
def retrieve_type(list):
    type_list = []
    for i in list:
        # print ("i is", i)
        type = nodes[i]['type']
        # print(tag)
        type_list.append(type)
    return (type_list)

""" Occupies the matched ground floor node with the data from the user"""
def occupy(nodes, x, req):
    nodes[x]['units'] = req['units']
    nodes[x]['tag'] = req['tag']
    nodes[x]['type'] = req['type']
    return nodes

""" De-occupies the ground floor node with the data from the user"""
def unoccupy(nodes, x, req):
    nodes[x]['units'] = None
    nodes[x]['tag'] = None
    nodes[x]['type'] = None
    return nodes

""" Occupies the matched second level node with the data from the user"""
def occupy2(nodes_2, x, req):
    nodes_2[x]['units'] = req['units']
    nodes_2[x]['tag'] = req['tag']
    nodes_2[x]['type'] = req['type']
    return nodes_2

""" De-occupies the second level node with the data from the user"""
def unoccupy2(nodes_2, x, req):
    nodes_2[x]['units'] = None
    nodes_2[x]['tag'] = None
    nodes_2[x]['type'] = None
    return nodes_2

#obtained definition from stack overflow, https://stackoverflow.com/questions/38987/how-do-i-merge-two-dictionaries-in-a-single-expression-taking-union-of-dictiona
def merge_two_dicts(x, y):
    z = x.copy()   # start with keys and values of x
    z.update(y)    # modifies z with keys and values of y
    return z

In [19]:
#FOUNDATION
# The aim of this script is to import all needed external data, like the results of the survey, the location data of nodes, and the edge information of the nodes.
# this data is imported and edited to be able to use efficiently and clearly throughout the whole code

"""Step 1: Reading JSON file from Rhino GH Export"""

#retrieve nodes.txt from Github (all nodes)
url = "https://raw.githubusercontent.com/erengozdeanil/Earthy4.2/main/0_Configuration/Foundation/nodes.txt"
resp = requests.get(url)
all_nodes = json.loads(resp.text)
#converts keys from str to int
all_nodes = {int(k) : v for k,v in all_nodes.items()}

#retrieve nodes_1.txt from Github (nodes on the ground floor)
url = "https://raw.githubusercontent.com/erengozdeanil/Earthy4.2/main/0_Configuration/Foundation/nodes_1.txt"
resp = requests.get(url)
nodes = json.loads(resp.text)
#converts keys from str to int
nodes = {int(k) : v for k,v in nodes.items()}

#retrieve nodes_2.txt from Github (nodes on the second floor)
url = "https://raw.githubusercontent.com/erengozdeanil/Earthy4.2/main/0_Configuration/Foundation/nodes_2.txt"
resp = requests.get(url)
nodes_2 = json.loads(resp.text)
#converts keys from str to int
nodes_2 = {int(k) : v for k,v in nodes_2.items()}

#retrieve edges.txt from Github
url = "https://raw.githubusercontent.com/erengozdeanil/Earthy4.2/main/0_Configuration/Foundation/edges.txt"
resp = requests.get(url)
edges = json.loads(resp.text)
#converts nested lists into a list of tuples
edges = [tuple(i) for i in edges]
print(nodes[0])

"""Step 2: Intitate Graph, add nodes & edges, determine adjacencies"""
G = nx.Graph()

k = list(nodes.keys())
G.add_nodes_from(k)

G.add_edges_from(edges)
# nx.draw_networkx(G)       #only required to draw if necessary

edge_dict = {}

adj = G.adjacency()

def create_edge_dict(graph):
    for i, n in G.adjacency():
        edge_dict[i] = list(n)
    return edge_dict

edge_dict = create_edge_dict(G)

"""Step 3: Finding and removing street nodes"""
street_nodes = []

for node, attr in nodes.items():
    #if "tag" = 0 (Local Street), 1 (Medium Street) or 2 (Main Street)
    if attr["tag"] == 0 or attr["tag"] == 1 or attr["tag"] == 2:
        street_nodes.append(node)

edge_dict_streetless = remove_street_node(nodes, edge_dict, street_nodes)


"""Step 4: Retrieve data from survey excel & simplifying data"""
from openpyxl import load_workbook
workbook = load_workbook(filename="Dummy_survey_data.xlsx")
workbook.sheetnames
sheet = workbook.active
sheet

# Create list with requests data
p_list=[]

#Simplify requests into:
#[
# district(1-12), 
# "D" - Daily OR "M" - Monthly OR "O" - Occasional, 
# "1" - Shop OR "2" - Shop+Workshop OR "3" - Workshop,
# units required: 1-9,
# name of occupant
# ]
for value in sheet.iter_rows(min_row=3,
                             min_col=6,
                             values_only=True):
    if value[3] != None and value[1] != None and value[2] != None:
        new_value=[int(value[2]), value[1], value[0], int(int(value[3])/9), value[4]]
        if new_value[2]=="Shop":
            new_value[2]="1"
        elif new_value[2]=="Workshop":
            new_value[2]="3"
        elif new_value[2]=="Both shop and workshop":
            new_value[2]="2"
        if new_value[1]=="Daily":
            new_value[1]="D"
        elif new_value[1]=="Monthly":
            new_value[1]="M"
        elif new_value[1]=="Occationally":
            new_value[1]="O"
    
        p_list.append(tuple(new_value))

p_list.sort()

# Create a dictionary from the survey data
user_req = {}

for i in p_list:
    value = {}
    value["name"] = p_list.index(i)
    value["district"] = i[0]
    value["use_frequency"] = i[1]
    value["type"] = i[2]
    value["units"] = i[3]
    value["tag"] = i[4]
    user_req[p_list.index(i)] = value


"""Step 5: Sorting the requirements based on District, Type, Use_Frequency, then Units required
This is to obtain the list in order of priority, from district, then by type (shops > shops+workshop > workshop), then
by use_frequency (Daily > Monthly > Occasional, and lastly by Units (Largest to smallest))
"""

dict_user_req = list(user_req.values())
user_req_list = sorted(dict_user_req, key=lambda d: (d['district'], d['type'], d['use_frequency'], -d['units']))

# create dictionairy from list again
sorted_user_req = {}

for item in user_req_list:
    sorted_user_req[item['name']] = item

print(sorted_user_req[1])


{'tag': 1, 'units': None, 'district': 7, 'name': 0, 'use_frequency': 'M', 'type': None}
{'name': 1, 'district': 7, 'use_frequency': 'D', 'type': '1', 'units': 1, 'tag': 'Carter'}


End of foundation code

In [20]:
# Organized vertical growth approach

""""Step 1: Determine the maximum depth of building layers on different streets"""

# Max. building extension perpendicular to street:
daily_layers = 3        # Local streets: max. of 3 units behind eachother (9 meter)
monthly_layers = 5      # Medium streets: max. of 5 units behind eachother (15 meter)
occationally_layers = 7 # Main streets: max. of 7 units behind eachother (21 meter)
unassigned_req = {}     # Dict containing all requests which are unassigned
failed = []             # List containing "key" of requests which are unassigned

""""Step 2: Match the user requests with the available nodes on the -ground floor-"""

# Match the user requests with the available nodes
for i, req in sorted_user_req.items():
    counter = 0
    unit_no = req["units"]
    req_frequency = req['use_frequency']
    req_district = req['district']

    # set layer to corresponding street type
    if req_frequency == 'D':
        layer = daily_layers
    elif req_frequency == 'M':
        layer = monthly_layers
    elif req_frequency == 'O':
        layer = occationally_layers

    current_node_list = []      # list of the nodes that have been occupied for one user
    current_node = None         # the node that has been place most recently
    occ = 0                     # the amount of units that have been placed
    failed_nodes = []           # the neighbor nodes that have been tried but didn't match         
    fail_count = 0              # the number of units that have been tried but didn't match
    
    # place the amount of units that is requested
    for count in range(unit_no):   
        # Step 1: Place first unit only on a layer 1 node when occ == 0, to make sure all buildings face the street
        if occ == 0:
            for x, node in nodes.items():
                #this should run for nodes that hasn't been already tested and failed 
                if node not in failed_nodes:
                    if req_frequency == node['use_frequency'] and req_district == node['district'] and node['tag'] == None and node['layer']==1:
                        current_node = x
                        current_node_list.append(x)
                        nodes = occupy(nodes, x, req)
                        occ += 1
                        break
        
        # Step 2: This part runs when more than 1 unit is requested
        else:            
            for cur_node in current_node_list:
                if occ != unit_no:
                    neighbour_list = list(G.neighbors(cur_node))
                    #check every neighbour, and find the best node to occupy
                    for neigh in neighbour_list:
                        #check if neighbour is:
                        
                        # A. Available
                        if nodes[neigh]['tag'] == None:
                            # Within the boundary of current street; neigh node['layer'] <= layer
                            if nodes[neigh]['layer'] <= layer:
                                if req_frequency == nodes[neigh]['use_frequency'] and req_district == nodes[neigh]['district']:
                                #Top criteria: to find the deeper node
                                    if  nodes[neigh]['layer'] == nodes[cur_node]['layer'] + 1:
                                        current_node_list.append(neigh)
                                        nodes = occupy(nodes, neigh, req)
                                        occ += 1
                                        break
                            
                        # B. If neighbor nodes 'tag' = 'x', this means that the border of the area is reached (max. distance from intersection) and we start placing at layer 1 nodes again
                        elif nodes[neigh]['tag'] == "x" and ((occ - layer) > -1):
                            neigh_1 = list(G.neighbors(current_node_list[occ - layer]))
                            for n in neigh_1:
                                if nodes[n]['tag'] == None and nodes[n]['layer'] == 1:
                                    current_node_list.append(n)
                                    nodes = occupy(nodes, n, req)
                                    occ += 1
                                    break
                                                    
                        # C. If neighbors position is > layers or not ''x' or occupied: De-occupy all nodes that has been assigned for this person and restart, occ == 0, append node to failed_nodes                 
                        else:
                            fail_count += 1
                        
                            #check: If all neighbours dont satisfy conditions, then deoccupy it
                            if fail_count == len(neighbour_list):
                                for j in current_node_list:
                                    unoccupy(nodes, j, req)
                                current_node_list = []
                                current_node = None
                                occ = 0
                                unassigned_req[i] = req
                                break
                                
                        # D. if neighbour is not available, JUST SKIP this neighbour, check next one

# Step 3: Establish requests that failed to find a place
for i in unassigned_req:
    failed.append(nodes[i]["name"])

# Measure how many requests have not been fullfilled to create second floor: only when more than 10 requests are to be placed
# And give summary of allocation process
number_unplaced = len(failed)
number_placed = len(list(sorted_user_req)) - number_unplaced

print("Number of placed requests:", number_placed)
print("Number of unplaced requests:", number_unplaced)
print("User index unplaced:", failed)




Number of placed requests: 92
Number of unplaced requests: 32
User index unplaced: [34, 32, 33, 30, 31, 29, 106, 99, 100, 101, 102, 103, 104, 105, 91, 92, 95, 96, 54, 55, 48, 49, 50, 53, 45, 47, 119, 116, 74, 75, 76, 72]


In [21]:
""""Step 3: Match the user requests with the available nodes on the -second floor-"""

# Max. building extension perpendicular to street:
daily_layers = 3        # Local streets: max. of 3 units behind eachother (9 meter)
monthly_layers = 5      # Medium streets: max. of 5 units behind eachother (15 meter)
occationally_layers = 7 # Main streets: max. of 7 units behind eachother (21 meter)
nodes_count_1 = 580     # nodes at second floor level start from here
unassigned_final = {}   # Dict containing all requests which are unassigned again

if number_unplaced > 10:
    # Match the user requests with the available nodes
    for i, req in unassigned_req.items():
        counter = 0
        unit_no = req["units"]
        req_frequency = req['use_frequency']
        req_district = req['district']

        # set layer to corresponding street type
        if req_frequency == 'D':
            layer = daily_layers
        elif req_frequency == 'M':
            layer = monthly_layers
        elif req_frequency == 'O':
            layer = occationally_layers

        current_node_list = []      # list of the nodes that have been occupied for one user
        current_node = None         # the node that has been place most recently
        occ = 0                     # the amount of units that have been placed
        failed_nodes2 = []           # the neighbor nodes that have been tried but didn't match         
        fail_count = 0              # the number of units that have been tried but didn't match
        
        # place the amount of units that is requested
        for count in range(unit_no):   
            # Step 1: Place first unit only on a layer 2 node when occ == 0, to make sure all buildings face the street
            if occ == 0:
                for x, node in nodes_2.items():
                    #this should run for nodes that hasn't been already tested and failed AND if the node below is occupied
                    if node not in failed_nodes2 and nodes[((node["name"]) - nodes_count_1)]["tag"] != None and nodes[((nodes_2[neigh]['name']) - nodes_count_1)]["tag"] != 3:
                        if req_frequency == node['use_frequency'] and req_district == node['district'] and node['tag'] == None and node['layer']==2:
                            current_node = x
                            current_node_list.append(x)
                            nodes_2 = occupy2(nodes_2, x, req)
                            occ += 1
                            break
            
            # Step 2: This part runs when more than 1 unit is requested
            else:            
                for cur_node in current_node_list:
                    if occ != unit_no:
                        neighbour_list = list(G.neighbors(cur_node))
                        #check every neighbour, and find the best node to occupy
                        for neigh in neighbour_list:
                            #check if neighbour is:
                            
                            # A. Available
                            if nodes_2[neigh]['tag'] == None:
                                # Within the boundary of current street; neigh node['layer'] <= layer
                                if nodes_2[neigh]['layer'] <= layer:
                                    if req_frequency == nodes_2[neigh]['use_frequency'] and req_district == nodes_2[neigh]['district']:
                                    #Top criteria: to find the deeper node
                                        if  nodes_2[neigh]['layer'] == nodes_2[cur_node]['layer'] + 1 and nodes[((nodes_2[neigh]['name']) - nodes_count_1)]["tag"] != None and nodes[((nodes_2[neigh]['name']) - nodes_count_1)]["tag"] != 3:
                                            current_node_list.append(neigh)
                                            nodes_2 = occupy2(nodes_2, neigh, req)
                                            occ += 1
                                            break
                                
                            # B. If neighbor nodes 'tag' = 'x', this means that the border of the area is reached (max. distance from intersection) and we start placing at layer 1 nodes again
                            elif nodes_2[neigh]['tag'] == "x" and ((occ - layer) > -1):
                                neigh_1 = list(G.neighbors(current_node_list[occ - layer]))
                                for n in neigh_1:
                                    if nodes_2[n]['tag'] == None and nodes_2[n]['layer'] == 1:
                                        current_node_list.append(n)
                                        nodes_2 = occupy2(nodes_2, n, req)
                                        occ += 1
                                        break
                                                        
                            # C. If neighbors position is > layers or not ''x' or occupied: De-occupy all nodes that has been assigned for this person and restart, occ == 0, append node to failed_nodes                 
                            else:
                                fail_count += 1
                            
                                #check: If all neighbours dont satisfy conditions, then deoccupy it
                                if fail_count == len(neighbour_list):
                                    for j in current_node_list:
                                        unoccupy2(nodes_2, j, req)
                                    current_node_list = []
                                    current_node = None
                                    occ = 0
                                    unassigned_final[i] = req
                                    break
                                    
                            # 2. if neighbour is not available, JUST SKIP this neighbour, check next one

failed_2 = []
print(unassigned_final)
for i in unassigned_final:
    failed_2.append(unassigned_req[i]["name"])
number_unplaced_2=len(failed_2)

print("Number of unplaced requests:", number_unplaced_2)
print("User identities:", failed_2)
print("\n")

{32: {'name': 32, 'district': 7, 'use_frequency': 'D', 'type': '1', 'units': 5, 'tag': 'Hallie'}, 33: {'name': 33, 'district': 7, 'use_frequency': 'D', 'type': '1', 'units': 5, 'tag': 'Sophie'}, 106: {'name': 106, 'district': 7, 'use_frequency': 'M', 'type': '1', 'units': 5, 'tag': 'Roy'}, 99: {'name': 99, 'district': 7, 'use_frequency': 'M', 'type': '1', 'units': 4, 'tag': 'Emily'}, 100: {'name': 100, 'district': 7, 'use_frequency': 'M', 'type': '1', 'units': 4, 'tag': 'Grace'}, 101: {'name': 101, 'district': 7, 'use_frequency': 'M', 'type': '1', 'units': 4, 'tag': 'Jamie'}, 102: {'name': 102, 'district': 7, 'use_frequency': 'M', 'type': '1', 'units': 4, 'tag': 'Jude'}, 103: {'name': 103, 'district': 7, 'use_frequency': 'M', 'type': '1', 'units': 4, 'tag': 'Lucas'}, 104: {'name': 104, 'district': 7, 'use_frequency': 'M', 'type': '1', 'units': 4, 'tag': 'Sienna'}, 105: {'name': 105, 'district': 7, 'use_frequency': 'M', 'type': '1', 'units': 4, 'tag': 'Willow'}, 54: {'name': 54, 'distri

In [22]:
"""Step 4: ____"""
for j in nodes_2:
    for i in nodes:
        if nodes[i]["tag"]==1:
            nodes_2[i+nodes_count_1]["tag"]==1
        elif nodes[i]["tag"]==2:
            nodes_2[i+nodes_count_1]["tag"]==2
        elif nodes[i]["tag"]==0: 
            nodes_2[i+nodes_count_1]["tag"]==0
            
"""Assigning "floor" attributes to all nodes"""

for k in nodes:
    nodes[k]['floor'] = None

for i in nodes:
    if (nodes_2[i+nodes_count_1]['tag'] == None) or (nodes_2[i+nodes_count_1]['tag'] == "x") or (nodes_2[i+nodes_count_1]['tag'] == 1) or (nodes_2[i+nodes_count_1]['tag'] == 2) or (nodes_2[i+nodes_count_1]['tag'] == 0) :
        # print(nodes_2[i+580])
        nodes[i]['floor'] = 0
    else:
        nodes[i]['floor'] = 1

for j in nodes_2:
    nodes_2[j]['floor'] = 0

"""Combining Ground and Level 1 nodes together and Exporting it together"""
nodes_final = merge_two_dicts(nodes, nodes_2)

file = "211024_nodes_occupied.txt"
with open(str(file), 'w') as outfile:
    try:
        json.dump(nodes_final, outfile)
        print(file + " has been updated successfully")
    except:
        print("Problem with updating file")

211024_nodes_occupied.txt has been updated successfully


End of foundation code